# 使用自动化机器学习

有多种可供你用于训练模型的机器学习算法，有时很难确定针对特定数据和预测需求的最有效算法。此外，通过使用标准化、缺失特征填补等技术对训练数据进行预处理，可以显著影响模型的预测性能。在寻找满足您的要求的最佳模型时，可能需要尝试多种算法和预处理转换的组合；这需要大量的时间和计算资源。

通过 Azure 机器学习，可以使用不同的算法和预处理选项来自动比较已训练的模型。你可以通过 [Azure 机器学习工作室](https://ml/azure.com)中的可视化界面或者 SDK 来使用此功能。使用此 SDK，可以更好地控制自动化机器学习试验的设置，不过可视化界面更易于使用。

## 准备工作

除了最新版本的 **azureml-sdk** 和 **azureml-widgets** 包之外，还需要 **azureml-train-automl** 包才能在此笔记本中运行代码。运行下面的单元以验证是否已安装它。

In [ ]:
!pip show azureml-train-automl

## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备数据

无需创建训练脚本来进行自动化机器学习，但需要加载训练数据。在本例中，你将使用一个包含糖尿病患者详细信息的数据集，然后将此数据集拆分为两个数据集：一个用于训练，另一个用于模型验证。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# Split the dataset into training and validation subsets
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

## 准备计算

云计算的好处之一是它可以按需缩放，使你能够预配足够的计算资源来并行处理自动化机器学习试验的多个子运行。

使用以下代码指定一个 Azure 机器学习计算群集（如果不存在，则会创建该群集）。

> **重要提示**：在运行以下代码之前，请先将代码中的 *your-compute-cluster* 更改为你的计算群集的名称！群集名称必须是长度在 2 到 16 个字符之间的全局唯一名称。有效字符是字母、数字和 - 字符。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **备注**：计算实例和群集是基于标准的 Azure 虚拟机映像。对于本练习，建议使用 *Standard_DS11_v2* 映像来实现成本和性能的最佳平衡。如果你的订阅配额不包含此映像，请选择其他映像；但请记住，较大的映像可能会产生更高的成本，但较小的映像可能不够完成任务。或者，可以请 Azure 管理员扩展你的配额。

## 配置自动化机器学习

现在可以配置自动化机器学习试验。

用于评估模型性能的指标是最重要的配置设置之一。你可以检索一个由自动化机器学习为特定类型的模型任务（分类或回归）计算的指标列表，例如：

In [ ]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

先决定想要优化的指标（本例中为 *AUC_weighted*），然后可以配置自动化机器学习运行。为此，你需要一个 AutoML 配置来指定目标指标以及要使用的数据、要尝试的组合数量等选项。

> **备注**：在此示例中，你将试验限制为 4 次迭代，以减少所需的时间。实际上，你可能会尝试更多的迭代。

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## 运行自动化机器学习试验

现在可以开始操作了。现在运行自动化机器学习试验。

> **备注**：这可能需要一些时间！

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-diabetes-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## 查看子运行详细信息

完成试验后，在小组件中查看输出，然后单击生成最佳结果的运行以查看其详细信息。

然后单击链接以在 Azure 机器学习工作室中查看试验详细信息并查看整体试验详细信息，然后再查看生成最佳结果的单个运行的详细信息。此处提供许多关于已生成模型的性能的信息。

使用此 SDK，你还可以检索所有子运行并查看它们的指标：

In [ ]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

## 获得最佳运行

你可以检索性能最佳的运行，并像这样查看其详细信息：

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

最后，找到性能最佳模型后就可以进行注册。

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **详细信息**：有关自动化机器学习的详细信息，请参阅 [Azure ML 文档](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)。